In [292]:
import logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(asctime)s - %(message)s')

import bs4
import requests
import pandas as pd
import numpy as np

import time
from io import StringIO
import os

import data_extraction as da
import modelling as md

In [ ]:
av_base_url = 'https://audiovault.net/movies'
df_list = []

for ii in range(1, 130):
    url = f'{av_base_url}?page={str(ii)}'
    response = requests.get(url)
    soup = bs4.BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table')
    df_list.append(pd.read_html(StringIO(str(table)))[0])
    time.sleep(0.5)

raw_df = pd.concat(df_list)

In [364]:
df = raw_df.copy()
df['year_primary'] = df['Name'].str.extract('\((\d{4})\)')[0]
df['year_secondary'] = df['Name'].str.extract('(\d{4})')[0]
df['year'] = np.where(df['year_primary'].notna(), df['year_primary'], df['year_secondary'])
df['year'] = np.where(df['year'].notna(), df['year'], 4000).astype(int)
df['cleaned_name'] = df['Name'].str.replace('\[[\w\s]*\]', '', regex=True).str.replace('\(\d{4}\)', '', regex=True).str.strip()
df = df[df.year.ge(2011)]
df = df.drop_duplicates(subset='cleaned_name')

df.to_parquet(os.path.join(da.audio_vault_dir, 'av_all_films.parquet'))

In [113]:
acb_base_url = 'https://www.classification.gov.au/titles/'
acb_df = da.get_acb_film_ratings()
acb_df['url_title'] = acb_df['url'].str.replace(acb_base_url, '').str.replace('\-\d{1,2}$', '', regex=True)

In [367]:
stop_words = ['the', 'of', 'a', 'in', 'up', 'at', 'is', 'to', 'by', 'this', 'an']
stop_char = ["'", ',', '.', '#']

In [368]:
acb_df['test_url'] = acb_df['movie'].str.lower().str.split(' ').apply(lambda x: '-'.join(y for y in x if not y in stop_words))

for char in stop_char:
    acb_df['test_url'] = acb_df['test_url'].str.replace(char, '')
acb_df['test_url'] = acb_df['test_url'].str.replace('-{2,}', '-', regex=True)

In [369]:
df['search_url'] = df['cleaned_name'].str.lower().str.split(' ').apply(lambda x: '-'.join(y for y in x if not y in stop_words))

for char in stop_char:
    df['search_url'] = df['search_url'].str.replace(char, '')
    df['search_url'] = df['search_url'].str.replace('-{2,}', '-', regex=True)

In [ ]:
def get_matrix_results(matrix, movie_info: dict):
    column_names = ['none', 'very mild', 'mild', 'moderate', 'strong', 'high']
    for ii, row in enumerate(matrix):
        # Find each of the 6 columns and iterate through those
        for jj, cell in enumerate(row.find_all("span", class_="impact-matrix__impact")):
            label_span = cell.find("span", class_="impact-matrix__rating-label")
            # When category gets film rating, they display the image of the classification icon (not just coloured circle)
            img_tag = cell.find("img")
            
            if label_span or img_tag:
                impact_column = column_names[jj]
        
        movie_info[md.full_cat_cols[ii]] = impact_column

    return movie_info


def get_metadata_table(soup, movie_info: dict):
    metadata_table = soup.find(class_="view-classification-title-nodes")

    if metadata_table is not None:

        for div in metadata_table.select(".views-field"):
            key = div.find("span", class_="views-label").get_text(strip=True)
            text = div.find("div", class_="field-content").get_text(strip=True)
            if text:
                movie_info[key] = text

    return movie_info


def check_eligibile_film(soup, movie_info: dict):

    access_denied = 'Access denied' in soup.text
    is_film = 'Category' in movie_info and movie_info['Category'] == 'Films'
    is_advertising = 'Category detail' in movie_info and 'Advertising' in movie_info['Category detail']
    is_current = False
    has_impact_table = len(soup.find_all("li", class_="impact-matrix__item")) in [6, 12]

    if 'Classification date' in movie_info and movie_info['Classification date'] != 'Not Specified':
        try:
            date = pd.to_datetime(movie_info['Classification date'])
            is_current = date > pd.to_datetime('2012-06-01')
        except:
            logging.info(f'Invalid datetime: {movie_info["Classification date"]}')

    found_eligible_film = False if access_denied else (is_film and is_current and has_impact_table and not is_advertising)

    return found_eligible_film

In [ ]:
record_list = []
outcome = []

for ii, search_url in enumerate(df['search_url']):

    if ii % 50 == 0:
        logging.info(f'{ii} / {df.shape[0]}')

    url = f'{acb_base_url}{search_url}'
    response = requests.get(url)
    soup = bs4.BeautifulSoup(response.content, 'html.parser')
    movie_info = {}

    if 'Page not found' in soup.text:
        record_list.append({})
        outcome.append('page-not-found')
        continue

    # Extract metadata
    movie_info = get_metadata_table(soup, movie_info)

    # If listing is from before additional data dates or is not a film (e.g. game), keep trying until you find one or run out
    # As there can be multiple listings with the same name
    jj = 0

    found_eligible_film = check_eligibile_film(soup, movie_info)
    while not found_eligible_film:
        time.sleep(1)
        url = f'{acb_base_url}{search_url}-{jj}'
        response = requests.get(url)
        soup = bs4.BeautifulSoup(response.content, 'html.parser')

        if 'Page not found' in soup.text:
            record_list.append({})
            outcome.append('no-eligible-film-found')
            break

        movie_info = get_metadata_table(soup, movie_info)
        found_eligible_film = check_eligibile_film(soup, movie_info)
        jj += 1

    if 'Page not found' in soup.text:
        continue

    movie_info['url'] = url

    # Find and iterate through 6 rows in classification table
    matrix = soup.find_all("li", class_="impact-matrix__item")

    # Check if page has impact table
    if matrix is None:
        record_list.append({})
        outcome.append('matrix-not-found')
        continue
    
    # Check size of impact table matches known patterns
    if len(matrix) == 6:
        movie_info = get_matrix_results(matrix, movie_info)
        outcome.append('success')
    elif len(matrix) == 12:
        # ACB tables can be doubled up due to mistakes in data
        movie_info = get_matrix_results(matrix[::2], movie_info)
        outcome.append('matrix-doubled-1')

        if all(val == 'none' for key, val in movie_info.items() if key in md.full_cat_cols):
            movie_info = get_matrix_results(matrix[1::2], movie_info)
            outcome[-1] = 'matrix-doubled-2'

    else:
        outcome.append('matrix-not-6-or-12')

    time.sleep(1)
    record_list.append(movie_info)

cat_df = pd.DataFrame(record_list)
cat_df['outcome'] = outcome

INFO - 2025-07-01 17:52:02,133 - 0 / 7048
INFO - 2025-07-01 17:52:51,451 - 50 / 7048
INFO - 2025-07-01 17:53:52,356 - 100 / 7048
INFO - 2025-07-01 17:54:47,895 - 150 / 7048
INFO - 2025-07-01 17:55:56,997 - 200 / 7048
INFO - 2025-07-01 17:56:49,485 - 250 / 7048
INFO - 2025-07-01 17:58:15,755 - 300 / 7048
INFO - 2025-07-01 18:00:23,070 - 350 / 7048
INFO - 2025-07-01 18:01:51,786 - 400 / 7048
INFO - 2025-07-01 18:03:46,891 - 450 / 7048
INFO - 2025-07-01 18:05:36,455 - 500 / 7048
INFO - 2025-07-01 18:07:13,751 - 550 / 7048
INFO - 2025-07-01 18:08:54,721 - 600 / 7048
INFO - 2025-07-01 18:10:31,509 - 650 / 7048
INFO - 2025-07-01 18:12:03,183 - 700 / 7048
INFO - 2025-07-01 18:14:09,770 - 750 / 7048
INFO - 2025-07-01 18:15:19,807 - 800 / 7048
INFO - 2025-07-01 18:17:10,415 - 850 / 7048
INFO - 2025-07-01 18:20:18,764 - 900 / 7048
INFO - 2025-07-01 18:22:04,611 - 950 / 7048
INFO - 2025-07-01 18:23:32,990 - 1000 / 7048
INFO - 2025-07-01 18:25:06,925 - 1050 / 7048
INFO - 2025-07-01 18:26:49,650 - 

In [404]:
cat_df['outcome'] = outcome
df = df.reset_index(drop=True)
df_w_ratings = pd.concat([df, cat_df], axis=1)
df_w_ratings.to_parquet(os.path.join('data', 'av_films_with_ratings.parquet'))

In [4]:
import pandas as pd
import os
df_w_ratings = pd.read_parquet(os.path.join('data', 'av_films_with_ratings.parquet'))

In [17]:
ratings_cols = ['Name', 'search_url', 'Classification', 'Consumer advice', 'Category', 'Category detail', 'Media type', 'Version', 'Duration',
       'Director/Creator', 'Producer', 'Production company',
       'Country of origin', 'Language', 'Applicant']

In [6]:
df_w_ratings.columns

Index(['ID', 'Name', 'Download', 'year_primary', 'year_secondary', 'year',
       'cleaned_name', 'search_url', 'Classification date',
       'Year of production', 'Classification', 'Consumer advice', 'Category',
       'Category detail', 'Media type', 'Version', 'Duration',
       'Director/Creator', 'Producer', 'Production company',
       'Country of origin', 'Language', 'Applicant', 'File number',
       'Classification number', 'Alternative titles', 'url', 'themes',
       'violence', 'language', 'drug_use', 'sex', 'nudity', 'Developer',
       'Publisher', 'Author', 'Reason for Refused Classification', 'outcome'],
      dtype='object')

In [ ]:
outside_classifiers = [
    'Film External Approved Decision received by API',
    'External API ',
    'Accredited Classifier Decision - Film',
    'Film - Netflix Classification Tool',
    'Film - Spherex Classification Tool',
    'Film - Amazon Classification Tool',
    'Decision made by Broadcaster',
    'Film Other',
]


In [9]:
df_w_ratings['Category detail'].value_counts()

Category detail
Film - Public Exhibition                           1321
Film - Sale/Hire                                   1093
Film External Approved Decision received by API     713
Public Exhibition                                   189
External API                                        149
Film Other                                           82
Accredited Classifier Decision - Film                79
Film - Netflix Classification Tool                   70
Film - Spherex Classification Tool                   58
Film - Amazon Classification Tool                    40
Advertising - Films                                  29
Film                                                 13
Decision made by Broadcaster                         10
Review - Public Exhibition                            8
Older Films                                           2
Review - Sale/Hire                                    2
CPE-W                                                 2
CFO-W                           

In [33]:
df_w_ratings[ratings_cols][df_w_ratings.Classification.eq('PG') & np.logical_not(df_w_ratings.Name.str.contains('Christ')) & df_w_ratings['Category detail'].eq('Film - Public Exhibition')].iloc[70:90]

,Name,search_url,Classification,Consumer advice,Category,Category detail,Media type,Version,Duration,Director/Creator,Producer,Production company,Country of origin,Language,Applicant
2148,Gambit (2012),gambit,PG,"Mild coarse language, violence and sexual refe...",Films,Film - Public Exhibition,DCP,ORIGINAL,89 minutes,MICHAEL HOFFMAN,MIKE LOBELL / ROB PARIS / ADAM RIPP,CBS FILMS,USA,ENGLISH,ROADSHOW FILM DISTRIBUTORS PTY LTD
2276,Goosebumps (2015),goosebumps,PG,Mild scary scenes,Films,Film - Public Exhibition,DCP,ORIGINAL,103 minutes,ROB LETTERMAN,DEBORAH FORTE / NEAL H. MORITZ,COLUMBIA PICTURES / SONY PICTURES ENTERTAINMEN...,USA,ENGLISH,ROADSHOW FILMS PTY LTD
2284,Grace Of Monaco (2014),grace-monaco,PG,"Mild themes, sexual reference and infrequent c...",Films,Film - Public Exhibition,DCP,ORIGINAL,103 minutes,OLIVIER DAHAN,UDAY CHOPRA / PIERRE-ANGE LE POGAM / ARASH AMEL,STONE ANGELS / YRF ENTERTAINMENT,"FRANCE, ITALY, USA","ENGLISH, FRENCH",ENTERTAINMENT ONE FILMS AUSTRALIA PTY LTD
2360,Hampstead (2017),hampstead,PG,Mild sexual references and coarse language,Films,Film - Public Exhibition,DCP,ORIGINAL,103 minutes,JOEL HOPKINS,ROBERT BERNSTEIN / DOUGLAS RAE,ECOSSE FILMS / MOTION PICTURE CAPITAL / SCOPE ...,UK,ENGLISH,ENTERTAINMENT ONE FILMS AUSTRALIA PTY LTD
2402,He Named Me Malala (2015),he-named-me-malala,PG,"Mild themes, some distressing images",Films,Film - Public Exhibition,DCP,ORIGINAL,87 minutes,DAVIS GUGGENHEIM,WALTER PARKES / LAURIE MACDONALD,IMAGE NATION / PARTICIPANT MEDIA,USA,ENGLISH,TWENTIETH CENTURY FOX FILM DIST. PTY LTD
2415,Heaven Is For Real (2014),heaven-for-real,PG,Mild themes,Films,Film - Public Exhibition,DCP,ORIGINAL,99 minutes,RANDALL WALLACE,JOE ROTH / T D JAKES,ROTH FILMS,USA,ENGLISH,SONY PICTURES RELEASING P/L
2419,Heidi (2015),heidi,PG,Mild themes,Films,Film - Public Exhibition,DCP,ORIGINAL,111 minutes,ALAIN GSPONER,JAKOB CLAUSSEN / LUKAS HOBI / ULI PUTZ / RETO ...,CLAUSSEN WOBKE PUTZ FILMPRODUKTION / ZODIAK PI...,"GERMANY, SWITZERLAND","GERMAN, SWISS GERMAN",STUDIOCANAL PTY LTD
2458,Hidden Figures (2016),hidden-figures,PG,Mild themes and coarse language,Films,Film - Public Exhibition,DCP,ORIGINAL,127 minutes,THEODORE MELFI,PETER CHERNIN / PHARRELL WILLIAMS / THEODORE M...,FOX 2000 PICTURES / CHERNIN ENTERTAINMENT / LE...,USA,ENGLISH,TWENTIETH CENTURY FOX FILM DIST. PTY LTD
2471,Hillsong - Let Hope Rise (2016),hillsong-let-hope-rise,PG,Mild themes,Films,Film - Public Exhibition,DVD,ORIGINAL,103 minutes,MICHAEL JOHN WARREN,JONATHAN BOCK / MATTHEW WEAVER / BEN FIELD,MEDIAWEAVER ENTERTAINMENT / GRACE HILL MEDIA /...,USA,ENGLISH,CROSSROAD DISTRIBUTORS
2518,Home (2015) [UK],home,PG,Mild threat,Films,Film - Public Exhibition,DCP 3D,ORIGINAL,94 minutes,TIM JOHNSON,MIREILLE SORIA / SUZANNE BUIRGY / CHRISTOPHER ...,DREAMWORKS ANIMATION,USA,ENGLISH,TWENTIETH CENTURY FOX FILM DIST. PTY LTD


In [408]:
df_w_ratings['Classification'].value_counts()

Classification
M                             1611
MA 15+                        1344
PG                             583
G                              223
R 18+                           70
Likely M                        17
Likely MA 15+                    5
Likely PG                        5
X 18+                            4
Likely G                         2
likely to be classified PG       2
RC                               1
Name: count, dtype: int64

In [ ]:
metadata_df[~metadata_df['Category detail'].isin(outside_classifiers)].iloc[20:]

,cleaned_name,Classification date,Year of production,Classification,Consumer advice,Category,Category detail,Media type,Version,Duration,Director/Creator,Producer,Production company,Country of origin,Language,Applicant,File number,Classification number,Alternative titles
28,Filth,10 September 2013,2013,R 18+,High impact sex scenes,Films,Film - Public Exhibition,DCP,ORIGINAL,97 minutes,JON S BAIRD,JON S BAIRD / WILL CLARKE / KEN MARSHALL / JAM...,STEEL MILL PICTURES / LOGIE PICTURES,UK,ENGLISH,ICON FILM DISTRIBUTION,T13/2976,259553,Not Specified
29,Flight,18 January 2013,2012,MA 15+,"Strong themes, drug use and nudity",Films,Film - Public Exhibition,DCP,ORIGINAL,138 minutes,ROBERT ZEMECKIS,WALTER F PARKES / LAURIE MACDONALD / ROBERT ZE...,NOT SHOWN,USA,ENGLISH,PARAMOUNT PICTURES AUSTRALIA PTY,T13/0221,256635,Not Specified
34,Guerrilla,4 July 2017,2017,MA 15+,"Strong themes, violence, drug use and coarse l...",Films,Film - Sale/Hire,DVD,ORIGINAL,312 minutes,JOHN RIDLEY / SAM MILLER,YVONNE ISIMEME IBAZEBO,SKY / SHOWTIME,UK,ENGLISH,RLJ ENTERTAINMENT AUSTRALIA PTY LTD,T17/2367,275191,Not Specified
35,Heaven Knows What,5 August 2015,2015,R 18+,High impact drug use,Films,Film - Sale/Hire,DVD,ORIGINAL,97 minutes,BENNY SAFDIE / JOSH SAFDIE,SEBASTIAN BEAR MCCLARD / OSCAR BOYSON,ICONOCLAST,USA,ENGLISH,ACCENT FILM ENTERTAINMENT,T15/1724,266749,Not Specified
36,Horns,28 January 2015,2013,R 18+,"High impact violence, blood and gore",Films,Film - Sale/Hire,DVD,ORIGINAL,138 minutes,ALEXANDRE AJA,CATHY SCHULMAN / RIZA AZIZ / JOEY MCFARLAND / ...,RED GRANITE PICTURES / MANDALAY PICTURES,USA,ENGLISH,ROADSHOW ENTERTAINMENT,T15/0171,264996,Not Specified
37,How High 2,10 April 2019,2019,MA 15+,Strong drug use,Films,Film - Sale/Hire,DVD,ORIGINAL,86 minutes,BRUCE LEDDY,JOSEPH P. GENIER / GREG HOLSTEIN,UNIVERSAL 1440 ENTERTAINMENT / SMITH-GARR PROD...,USA,ENGLISH,UNIVERSAL SONY PICTURES HOME ENTERTAINMENT AUSTRA,T19/0993,283871,Not Specified
38,Ill Manors,18 March 2013,2012,R 18+,High impact coarse language and drug use,Films,Film - Sale/Hire,DVD,ORIGINAL,141 minutes,BEN DREW,ATIF GHANI,FILM LONDON MICROWAVE,UK,ENGLISH,PINNACLE FILMS PTY LTD,T13/0605,257043,Not Specified
40,Inherent Vice,13 January 2015,2014,MA 15+,"Strong sex scenes, frequent drug use and nudity",Films,Film - Public Exhibition,DCP,ORIGINAL,149 minutes,PAUL THOMAS ANDERSON,JOANNE SELLAR / DANIEL LUPI / PAUL THOMAS ANDE...,JOANNE SELLAR / GHOULARDI FILM COMPANY,USA,ENGLISH,ROADSHOW FILMS PTY LTD,T15/0039,264844,Not Specified
41,Kill Your Friends,4 July 2016,2015,MA 15+,"Strong coarse language, violence, sex and drug...",Films,Film - Sale/Hire,DVD,ORIGINAL,99 minutes,OWEN HARRIS,LEN BLAVATNIK / GREGOR CAMERON / WILL CLARKE,UNIGRAM / ALTITUDE FILM ENTERTAINMENT,UK,ENGLISH,ROADSHOW ENTERTAINMENT,T16/1793,270555,Not Specified
44,Mandy,17 August 2018,2017,MA 15+,"Strong horror themes, bloody violence, drug us...",Films,Film - Public Exhibition,DCP,ORIGINAL,121 minutes,PANOS COSMATOS,DANIEL NOAH / JOSH C. WALLER / ELIJAH WOOD / N...,"SPECTREVISION, UMEDIA AND XYZ FILMS",USA,ENGLISH,MADMAN ENTERTAINMENT,T18/3309,281078,Not Specified


In [ ]:
# TODO: Extract consumer advice
# TODO: verify acb_df entries

# TODO: get category (classified by gov or accredited classifier)
# Then retry with incremented number if Category: Computer Games etc

#consumer advice: "views-field views-field-field-consumer-advice"
#category: views-field views-field-field-category (Films)
#category detail: views-label views-label-field-category-detail (Films - Public Exhibition)
#applicant: views-label views-label-field-applicant (Netflix)